In [1]:
#sample data
#each row is an example
#the first two columns are features
#the last row is the label

training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red' , 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

     
    

In [2]:
#adding column labels
header = ['color', 'diameter', 'label']

def unique_vals(rows, cols):
    return set([row[cols] for row in rows])

#example
#unique_vals(training_data, 0) -> finding uinque vals in terms of color
#unique_vals(training_data, 1) -> finding uinque vals in terms of diameter

In [3]:

def class_counts(rows):
    counts  = {}
    for row in rows:
        label = row[-1] # as our label is in the last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [4]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)


In [5]:
'''A question is used to partition the dataset. 
this class records the 'column number'(0 for colour) and a 
'column value' (green). The 'match' methood is used to compare
the feature value in an example to the feature value stored in 
the question.
'''

class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):   #helper method to print the que in readable format
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return 'Is %s %s %s?' % (
            header[self.column], condition, str(self.value))

In [6]:
"""this method is use to partition the dataset
for each row in the dataset, it check is it matches the question
if it does it adds it to 'true rows' or to 'false rows'
"""

def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

"""lets partition the trainig data based on whether rows are red
true_rows, false_rows = partition(training_data, Question(0, 'red'))
it will contain all the red rows  <- true_rows
"""
    


"lets partition the trainig data based on whether rows are red\ntrue_rows, false_rows = partition(training_data, Question(0, 'red'))\nit will contain all the red rows  <- true_rows\n"

# Decisison Tree Algo Imlementation

In [7]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_label = counts[label]/float(len(rows))
        impurity -= prob_label**2
    return impurity

In [8]:
def info_gain(left, right, current_uncertainity):
    p = float(len(left))/(len(left)+len(right))
    return current_uncertainity - p*gini(left) - (1-p)*gini(right)

In [9]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col,val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
                
    return best_gain, best_question

In [10]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
class Decision_Node:
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [11]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain ==0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [12]:
def print_tree(node, spacing=''):
    if isinstance(node, Leaf):
        print(spacing+"predict", node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + '--> True :')
    print_tree(node.true_branch, spacing + " ")
    print(spacing + '--> False :')
    print_tree(node.false_branch, spacing + " ")



In [13]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [14]:
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs ={}
    for label in counts.keys():
        probs[label] = str(int(counts[label]/total*100))+'%'
    return probs

In [15]:
if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 3, 'Mango'],
        ['Red' , 1, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon']
    ]
    
    for row in testing_data:
        print('Actual : %s. Predicted : %s' %
              (row[-1], print_leaf(classify(row, my_tree))))
    

Is diameter >= 3?
--> True :
 Is color == Yellow?
 --> True :
  predict {'Mango': 1, 'Lemon': 1}
 --> False :
  predict {'Mango': 1}
--> False :
 predict {'Grape': 2}
Actual : Mango. Predicted : {'Mango': '100%'}
Actual : Mango. Predicted : {'Mango': '50%', 'Lemon': '50%'}
Actual : Grape. Predicted : {'Grape': '100%'}
Actual : Grape. Predicted : {'Grape': '100%'}
Actual : Lemon. Predicted : {'Mango': '50%', 'Lemon': '50%'}
